## 1. Importing the Data
From:
<ol>
<li><code>"datasets/airbnb_price.csv"</code></li>
<li><code>"datasets/airbnb_room_type.xlsx"</code></li>
<li><code>"datasets/airbnb_last_review.tsv"</code></li>
</ol>

In [1]:
import pandas as pd
import numpy as np
import datetime as dt

prices = pd.read_csv('/content/airbnb_price.csv')
xls = pd.ExcelFile('/content/airbnb_room_type.xlsx')

room_types = xls.parse(0)
reviews = pd.read_csv("/content/airbnb_last_review.tsv", sep="\t")

print(prices.head(), "\n", room_types.head(), "\n", reviews.head())

   listing_id        price                nbhood_full
0        2595  225 dollars         Manhattan, Midtown
1        3831   89 dollars     Brooklyn, Clinton Hill
2        5099  200 dollars     Manhattan, Murray Hill
3        5178   79 dollars  Manhattan, Hell's Kitchen
4        5238  150 dollars       Manhattan, Chinatown 
    listing_id                                description        room_type
0        2595                      Skylit Midtown Castle  Entire home/apt
1        3831            Cozy Entire Floor of Brownstone  Entire home/apt
2        5099  Large Cozy 1 BR Apartment In Midtown East  Entire home/apt
3        5178            Large Furnished Room Near B'way     private room
4        5238         Cute & Cozy Lower East Side 1 bdrm  Entire home/apt 
    listing_id    host_name   last_review
0        2595     Jennifer   May 21 2019
1        3831  LisaRoxanne  July 05 2019
2        5099        Chris  June 22 2019
3        5178     Shunichi  June 24 2019
4        5238          

## 2. Cleaning the price column
Steps:
<ol>
<li>calculating the average price per listing by <code>room_type</code></li>
<li>cleaning the <code>price</code> column (removing 'dollars' string) in order to calculate the average price</li>
</ol>

In [2]:
prices["price"] = prices["price"].str.replace(" dollars", "")
prices["price"] = pd.to_numeric(prices['price'])

print(prices['price'].describe())

count    25209.000000
mean       141.777936
std        147.349137
min          0.000000
25%         69.000000
50%        105.000000
75%        175.000000
max       7500.000000
Name: price, dtype: float64


## 3. Calculating average price
<li>Three quarters of listings cost \$175 per night or less</li>
<li>There are some outliers including a maximum price of \$7,500 per night!</li>
<li>Some of listings show up as free</li>

In [3]:
free_listings = prices["price"] == 0
prices = prices.loc[~free_listings]
avg_price = round(prices['price'].mean(),2)

print("The average price per night for an Airbnb listing in NYC is ${}.".format(avg_price))

The average price per night for an Airbnb listing in NYC is $141.82.


## 4. Comparing costs to the private rental market
<p>According to <a href="https://www.zumper.com/rent-research">Zumper</a>, a 1 bedroom apartment in New York City costs, on average, $3,100 per month. The per night prices of listings into monthly costs is needed to compare to the private market. </p>

In [4]:
prices["price_per_month"] = prices["price"] * 365/12
average_price_per_month = round(prices['price_per_month'].mean(),2)

print("airbnb monthly costs are ${}, while in the private market you would pay {}.".format(average_price_per_month,"$3,100.00"))

airbnb monthly costs are $4313.61, while in the private market you would pay $3,100.00.


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


## 5. Cleaning the room type column
<p>To find out the breakdown of listings by type of room... The <code>room_type</code> column has several variations for <code>private room</code> listings, because of different case (upper and lower).

In [5]:
room_types["room_type"] = room_types["room_type"].str.lower()
room_types["room_type"] = room_types["room_type"].astype('category')
room_frequencies = room_types['room_type'].value_counts()

print(room_frequencies)

entire home/apt    13266
private room       11356
shared room          587
Name: room_type, dtype: int64


## 6. What timeframe are we working with?
<p>Observation: There is a fairly similar sized market opportunity for both private rooms (45% of listings) and entire homes/apartments (52%) on the Airbnb platform in NYC.</p>
<p>Also, the <code>last_review</code> column contains the date of the last review in the format of "Month Day Year" e.g., May 21 2019. To find out the earliest and latest review dates in the DataFrame, and ensuring the format allows this analysis to be easily conducted going forwards. </p>

In [6]:
reviews["last_review"] = pd.to_datetime(reviews['last_review'])
first_reviewed = reviews["last_review"].dt.date.min()
last_reviewed = reviews["last_review"].dt.date.max()

print("The latest Airbnb review is {}, the earliest review is {}".format(first_reviewed,last_reviewed))

The latest Airbnb review is 2019-01-01, the earliest review is 2019-07-09


## 7. Joining the DataFrames.
<p>After joining the data, any observations with missing values and duplicates are removed.</p>

In [7]:
rooms_and_prices = prices.merge(room_types, how='outer', on='listing_id')
airbnb_merged = rooms_and_prices.merge(reviews, how='outer', on='listing_id')
airbnb_merged.dropna(inplace=True)

print("There are {} duplicates in the DataFrame.".format(airbnb_merged.duplicated().sum()))

There are 0 duplicates in the DataFrame.


## 8. Analyzing listing prices by NYC borough
Boroughs are listed as the first part of a string within the <code>nbhood_full</code> column, e.g.,
<pre><code>Manhattan, Midtown
Brooklyn, Clinton Hill
Manhattan, Murray Hill
Manhattan, Hell's Kitchen
Manhattan, Chinatown</code></pre>


In [8]:
airbnb_merged["borough"] = airbnb_merged["nbhood_full"].str.partition(",")[0]
boroughs = airbnb_merged.groupby("borough")["price"].agg(["sum", "mean", "median", "count"])
boroughs = boroughs.round(2).sort_values("mean", ascending=False)

print(boroughs)

                     sum    mean  median  count
borough                                        
Manhattan      1898417.0  184.04   149.0  10315
Brooklyn       1275250.0  122.02    95.0  10451
Queens          320715.0   92.83    70.0   3455
Staten Island    22974.0   86.04    71.0    267
Bronx            55156.0   79.25    65.0    696


## 9. Price range by borough
<p>To categorize listings based on whether they fall into specific price ranges, and view this by borough:
<br>
Used percentiles and labels to create a new column, <code>price_range</code>, in the DataFrame.
Grouped the data and count frequencies for listings in each price range by borough; 
And assigned the following categories and price ranges:</p>
<table>
<thead>
<tr>
<th>label</th>
<th>price</th>
</tr>
</thead>
<tbody>
<tr>
<td><code>Budget</code></td>
<td>\$0-69</td>
</tr>
<tr>
<td><code>Average</code></td>
<td>\$70-175</td>
</tr>
<tr>
<td><code>Expensive</code></td>
<td>\$176-350</td>
</tr>
<tr>
<td><code>Extravagant</code></td>
<td>&gt; \$350</td>
</tr>
</tbody>
</table>

In [9]:
label_names = ["Budget", "Average", "Expensive", "Extravagant"]
ranges = [0, 69, 175, 350, np.inf]
airbnb_merged["price_range"] = pd.cut(airbnb_merged["price"], bins=ranges, labels=label_names)

prices_by_borough = airbnb_merged.groupby(["borough", "price_range"])["price_range"].count()
print(prices_by_borough)

borough        price_range
Bronx          Budget          381
               Average         285
               Expensive        25
               Extravagant       5
Brooklyn       Budget         3194
               Average        5532
               Expensive      1466
               Extravagant     259
Manhattan      Budget         1148
               Average        5285
               Expensive      3072
               Extravagant     810
Queens         Budget         1631
               Average        1505
               Expensive       291
               Extravagant      28
Staten Island  Budget          124
               Average         123
               Expensive        20
               Extravagant       0
Name: price_range, dtype: int64
